In [1]:
# Inicializamos el Tensorflow para no consumir toda la memoria
# de la GPU de golpe (está bien para modelos pequeños)
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [2]:
# Importamos nuestra librerías básicas
import numpy as np
import os
import itertools
from time import time

from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import TensorBoard
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns

# Este paso es para que los resultados en vuestros notebooks sean iguales a lo de este
np.random.seed(42)

Using TensorFlow backend.


# Aprendizaje supervisado - Revisión
Hemos visto como utilizar las técnicas de aprendizaje automatico que no son basadas en redes neuronales. En todas ellas, sin practicamente optimización en los parametros por defecto del sklearn, hemos intentado clasificar el dataset MNIST.

<table style="width:100%">
  <tr>
    <th>Algoritmo</th>
    <th>Exactitud entrenamiento</th> 
    <th>Exactitud cross-validación</th>
    <th>Exactitud pruebas</th>
    <th>Error</th>
    <th>Velocidad entrenamiento</th>
    <th>Velocidad predicción</th>
    <th>Comentarios</th>
  </tr>
  <tr>
    <td>Regresión Logistica</td>
    <td>89.95%</td> 
    <td>88.22%</td>
    <td>89.45%</td>
    <td>10.55%</td>
    <td>12.3s</td>
    <td>24.6ms/10.500 predicción</td>
    <td>Solo 7.000 instancias entrenadas (lento para entrenar)</td>
  </tr>
  <tr>
    <td>SVM, kernel lineal</td>
    <td>100%</td> 
    <td>-</td>
    <td>89.52%</td>
    <td>10.48%</td>
    <td>50s</td>
    <td>2.26s/1.050 predicción</td>
    <td></td>
  </tr>
  <tr>
    <td>SVM, kernel RBF</td>
    <td>97.96%</td> 
    <td>-</td>
    <td>93.62%</td>
    <td>6.38%</td>
    <td>1min52s</td>
    <td>3.61s/1.050 predicción</td>
    <td>Más lento que el anterior</td>
  </tr>
  <tr>
    <td>KNN</td>
    <td>100%</td> 
    <td>97.23%</td>
    <td>97.51%</td>
    <td>2.49%</td>
    <td>26.2s</td>
    <td>3min26s/10.500 predicción</td>
    <td>¡50ms por predicción!</td>
  </tr>
  <tr>
    <td>Decision Tree</td>
    <td>100%</td> 
    <td>86.68%</td>
    <td>87.57%</td>
    <td>12.43%</td>
    <td>22s</td>
    <td>24.1ms/10.500 predicción</td>
    <td></td>
  </tr>
  <tr>
    <td>Decision Tree ("optimizada")</td>
    <td>98.96%</td> 
    <td>87.02%</td>
    <td>88.12%</td>
    <td>11.88%</td>
    <td>15.6s</td>
    <td>27ms/10.500 predicción</td>
    <td>Tras optimizar con GridSearchCV (4min37s)</td>
  </tr>
  <tr>
    <td>Random Forest</td>
    <td>99.91%</td> 
    <td>94.29%</td>
    <td>95.02%</td>
    <td>4.98%</td>
    <td>3.93s</td>
    <td>62ms/10.500 predicción</td>
    <td></td>
  </tr>
  <tr>
    <td>Regresión Logistica con Bagging</td>
    <td>89.03%</td> 
    <td>88.36%</td>
    <td>89.54%</td>
    <td>10.46%</td>
    <td>2min19s</td>
    <td>1.05s/10.500 predicción</td>
    <td></td>
  </tr>
  <tr>
    <td>Decision Tree con Boosting</td>
    <td>84.54%</td> 
    <td>83.79%</td>
    <td>84.82%</td>
    <td>15.18%</td>
    <td>4min9s</td>
    <td>113ms/10.500 predicción</td>
    <td></td>
  </tr>
  <tr>
    <td>Stacking (1a Log Reg + Rand Forest, 2a Rand Forest)</td>
    <td>-</td> 
    <td>-</td>
    <td>95.53%</td>
    <td>4.47%</td>
    <td>1min9s</td>
    <td>-</td>
    <td></td>
  </tr>
</table>


# Aprendizaje Supervisado - Redes Neuronales
La neurona
![Neurona](images\Neurona.png)
> https://themenwhostareatcodes.wordpress.com/2014/03/02/neural-networks-in-a-nutshell/

### Conceptos clave

- $\sum$ - es la función a aplicar a las entradas, tipicamente la suma, de las entradas ponderada por el peso $w_{km}$, adicionando un termo de bias $b_k$
- $k,m$ - en este caso se refieren al peso asociado con la entrada $m$ de la neurona $k$ de la red neuronal
- Los pesos $w_{km}$ se inicializan a principio, antes del proceso de aprendizaje (función de inicialización) de forma *aleatoria*
- $\varphi$ - es la función de activación, i.e. que hace la neurona enviar una salida y cuanto es. Una función que era muy comun era la función logística que se utiliza en la regresión logística  $\sigma(v_k) = \frac{1}{1+e^{-v_k}}$
- La salida $y_k$ se compara con el resultado esperado y la diferencia es la *loss function*. El objetivo del aprendizaje es minimar esta función

### *Forward pass*

### *Backpropagation*


In [ ]:
# Aqui importamos los datos que vamos a clasificar

# Scikit-Learn ya incluye algunos datasets de ejemplo como el MNIST
mnist = fetch_mldata('MNIST original')

# Aqui cargamos nuestros ejemplos en X, el target en y. Nuestro objetivo con
# Machine Learning es aprender la función f(X) que genera y.
X, y = mnist["data"], mnist["target"]

In [ ]:
# Nuestro objetivo es que la función aprendida funcione no solamente con
# el dataset de prueba, pero que también "generalize" bien para ejemplos
# que no haya visto antes.

# En esta sección separamos el dataset en 2 partes: el training set y test set
X_train_o, X_test_o, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
# Normalizamos los datos, ya que eso es necesario para las redes neuronales
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_o)
X_test = scaler.transform(X_test_o)

## Clasificador con Redes Neuronales sencilla

In [ ]:
# Creamos el modelo de red neuronal que queremos utilizar
def model_func():
    # Sequential indica que cada capa se ejecuta una detras de otra
    model = Sequential()
    # En las redes neuronales es particularmente importante inicializar correctamente los
    # parametros. Esto permite llegar a un modelo estable rápidamente
    # hemos seleccionado 1000 neuronas de forma arbitraria. Además utilizamos ReLU como
    # activacion.
    model.add(Dense(1000, kernel_initializer='he_normal', bias_initializer='he_normal', activation='relu', input_dim=784))
    # A continuación creamos la capa de salida que tiene las 10 categorías
    model.add(Dense(10, kernel_initializer='he_normal', bias_initializer='he_normal'))
    # Finalmente aplicamos una activación softmax que básicamente garantiza que la salida de esta capa
    # es una probabilidad distribuida en las 10 salidas de la red
    model.add(Activation('softmax'))
    # Aquí definimos cual es la función que queremos minimizar (esto es el error de clasificación
    # de cada categoría) y el algoritmo de aprendizaje (que en este caso es el Nesterov-Adam)
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

In [ ]:
# Tensorflow incluye el Tensorboard que permite seguir el progreso del aprendizaje del modelo.
# Como estamos utilizando un modelo complejo, el Tensorboard nos permitirá ver si el modelo está
# progresando correctamente. En el caso que no, lo podríamos volver a arrancar.
tensorboard=TensorBoard(log_dir="logs/{}".format(time()))
# Al definir el modelo decimos cuantas veces queremos mirar los datos (aquí hemos incluido
# 1000 veces) además de enviar 200 ejemplos de training de cada vez (en vez de 1 en 1 del
# SGD). Esto sería un Mini-batch SGD.    
nn_clf = KerasClassifier(build_fn=model_func, callbacks=[tensorboard], epochs=50, batch_size=1000, verbose=0)
# Hacemos fit del modelo al dato
%time nn_clf.fit(X_train, y_train)

In [ ]:
# Medimos el rendimiento
%time train_score = nn_clf.score(X_train, y_train)
print("Rendimiento en el dataset de training: %.4f" % train_score)
%time score = nn_clf.score(X_test, y_test)
print("Rendimiento en el dataset de pruebas: %.4f" % score)

In [ ]:
# Esta función esta adaptada de la documentación del Scikit-Learn para presentar
# la matrix de "confusión" de forma más visual
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    #else:
        #print('Confusion matrix, without normalization')
        

    #print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel("Valor real")
    plt.xlabel("Predicción")
    plt.show()

In [ ]:
# La confusion matrix nos presenta una información sumaria de la precisión y del recall
cm=confusion_matrix(y_test, nn_clf.predict(X_test))
plt.figure(figsize=(8,8))
plot_confusion_matrix(cm, classes=["zero","uno","dos","tres","cuatro","cinco","seis","siete","ocho","nueve"], normalize=True);